In [1]:
# In[]
from random import randint
from numpy import array
from pickle import dump
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
import numpy as np
# In[]

import string
import re
# turn a doc into clean tokens
def clean_doc(doc):
    # replace '--' with a space ' '
    doc = doc.replace('--', ' ')
    # split into tokens by white space
    tokens = doc.split()
    # remove punctuation from each token
    tokens = [re.sub(r'[^\w\s]','',w) for w in tokens]
    # remove remaining tokens that are not alphabetic
    tokens = [word for word in tokens if word.isalpha()]
    # make lower case
    tokens = [word.lower() for word in tokens]
    return tokens


# In[]
from nltk.corpus import gutenberg

tokens_raw1 = gutenberg.words('austen-emma.txt')
n1 = len(tokens_raw1)
tokens_train1  = tokens_raw1[0:int(0.8*n1)];
tokens_test1 = tokens_raw1[int(0.8*n1):n1]; 

tokens_raw2 = gutenberg.words('austen-sense.txt')
n2 = len(tokens_raw2)
tokens_train2  = tokens_raw2[0:int(0.8*n2)];
tokens_test2 = tokens_raw2[int(0.8*n2):n2]; 

tokens_raw = tokens_train1 + tokens_train2
tokens_test = tokens_test1 + tokens_test2

text_raw = ' '.join(tokens_raw)
tokens = clean_doc(text_raw)



len(tokens_test)


/home/chaikesh/anaconda2/envs/tensorflow/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


66802

In [2]:
# In[]
# organize into sequences of tokens
length = 50 + 1
step =2
sequences = list()
for i in range(0,len(tokens)-length, step):
	# select sequence of tokens
	seq = tokens[i:i+length]
	# convert into a line
	line = ' '.join(seq)
	# store
	sequences.append(line)
print('Total Sequences: %d' % len(sequences))

Total Sequences: 112954


In [3]:
lines = sequences
# In[]
# integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)

# In[]

# vocabulary size
vocab_size = len(tokenizer.word_index) + 1

# In[]
# separate into input and output
sequences = array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]
seq_length

50

In [4]:
# define model
model = Sequential()
model.add(Embedding(vocab_size, 300, input_length=seq_length))
model.add(LSTM(50, return_sequences=True))
model.add(LSTM(50))
model.add(Dense(50, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

# In[]

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 300)           2483400   
_________________________________________________________________
lstm_1 (LSTM)                (None, 50, 50)            70200     
_________________________________________________________________
lstm_2 (LSTM)                (None, 50)                20200     
_________________________________________________________________
dense_1 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_2 (Dense)              (None, 8278)              422178    
Total params: 2,998,528
Trainable params: 2,998,528
Non-trainable params: 0
_________________________________________________________________
None


In [6]:
# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
model.fit(X, y, batch_size=128, epochs=60)

# In[]

# save the model to file
model.save('model_a03_wl_300_50_80.h5')
# save the tokenizer
dump(tokenizer, open('tokenizer_a03_wl_300_50_80.pkl', 'wb'))

Epoch 1/60
45182/45182 [==============================] - 223s 5ms/step - loss: 6.6680 - acc: 0.0321
Epoch 2/60
45182/45182 [==============================] - 220s 5ms/step - loss: 6.1848 - acc: 0.0365
Epoch 3/60
45182/45182 [==============================] - 221s 5ms/step - loss: 6.0543 - acc: 0.0463
Epoch 4/60
45182/45182 [==============================] - 221s 5ms/step - loss: 5.9393 - acc: 0.0551
Epoch 5/60
45182/45182 [==============================] - 221s 5ms/step - loss: 5.8256 - acc: 0.0611
Epoch 6/60
45182/45182 [==============================] - 220s 5ms/step - loss: 5.7212 - acc: 0.0668
Epoch 7/60
45182/45182 [==============================] - 220s 5ms/step - loss: 5.6301 - acc: 0.0718
Epoch 8/60
45182/45182 [==============================] - 219s 5ms/step - loss: 5.5427 - acc: 0.0770
Epoch 9/60
45182/45182 [==============================] - 219s 5ms/step - loss: 5.4553 - acc: 0.0837
Epoch 10/60
45182/45182 [==============================] - 220s 5ms/step - loss: 5.3708 - a

In [6]:
from random import randint
from pickle import load
from keras.models import load_model
# load the model
model = load_model('model_a03_wl_300_50_80.h5')
# load the tokenizer
tokenizer = load(open('tokenizer_a03_wl_300_50_80.pkl', 'rb'))

In [7]:
# In[]
from keras.preprocessing.sequence import pad_sequences


# generate a sequence from a language model
def generate_seq(model, tokenizer, seq_length, seed_text, n_words):
	result = list()
	in_text = seed_text
	# generate a fixed number of words
	for _ in range(n_words):
		# encode the text as integer
		encoded = tokenizer.texts_to_sequences([in_text])[0]
		# truncate sequences to a fixed length
		encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
		# predict probabilities for each word
		yhat = model.predict_classes(encoded, verbose=0)
        #yhat1 = model.predict(encoded, verbose=0)
		# map predicted word index to word
		out_word = ''
		for word, index in tokenizer.word_index.items():
			if index == yhat:
				out_word = word
				break
		# append to input
		in_text += ' ' + out_word
		result.append(out_word)
	return ' '.join(result)

In [8]:
# select a seed text
seed_text = lines[randint(0,len(lines))]
print(seed_text + '\n')
generated = generate_seq(model, tokenizer, seq_length, seed_text, 15)
print(generated)


t class us together harriet my playing is no more like her s than a lamp is like sunshine oh dear i think you play the best of the two i think you play quite as well as she does i am sure i had much rather hear you every body

but i should not be tempted for she had often called at his son and


In [20]:
#perplexity

test_tokens = tokens_test[1:1000]


in_text = test_tokens[0]
logp = 0
c=0
for word in test_tokens:
    encoded = tokenizer.texts_to_sequences([in_text])[0]
    encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
    print encoded
    yhat1 = model.predict(encoded,verbose=0)[0]
    next_word = word
    if next_word in tokenizer.word_index:
        p = yhat1[tokenizer.word_index[next_word]]
        if  p>=0.00001:
            c+=1
            logp = logp + np.log(p)

    else:
        p = 1
        c+=1
        logp = logp + np.log(p)
    
    in_text +=' ' + next_word
pplxty = np.exp(-1*logp/c)

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0 28]]
[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  28 25]]
[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 28
  25  1]]
[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 28 25
   1 90]]
[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0 

[[   0    0    0    0    0    0    0   28   25    1   90 1442    7  597
   168   18    1  191  889 5124  104  311    3  105   58   43   35  663
  2450 2451   96 5474    1 1843   41    9    1  946 2237    4    1  221
    31  475    9 1877  102    7   16  123]]
[[   0    0    0    0    0    0   28   25    1   90 1442    7  597  168
    18    1  191  889 5124  104  311    3  105   58   43   35  663 2450
  2451   96 5474    1 1843   41    9    1  946 2237    4    1  221   31
   475    9 1877  102    7   16  123   11]]
[[   0    0    0    0    0   28   25    1   90 1442    7  597  168   18
     1  191  889 5124  104  311    3  105   58   43   35  663 2450 2451
    96 5474    1 1843   41    9    1  946 2237    4    1  221   31  475
     9 1877  102    7   16  123   11   19]]
[[   0    0    0    0    0   28   25    1   90 1442    7  597  168   18
     1  191  889 5124  104  311    3  105   58   43   35  663 2450 2451
    96 5474    1 1843   41    9    1  946 2237    4    1  221   31  475
    

[[ 475    9 1877  102    7   16  123   11   19   53   65   11    9  424
     2    3   15   11  190    8   27   91  105   11   33   12   85    8
     7  296  168 1323    7  223  182   16   30   12   20  910    9 3825
     2    5 1343 3904 1159  120    2   61]]
[[   9 1877  102    7   16  123   11   19   53   65   11    9  424    2
     3   15   11  190    8   27   91  105   11   33   12   85    8    7
   296  168 1323    7  223  182   16   30   12   20  910    9 3825    2
     5 1343 3904 1159  120    2   61  148]]
[[   9 1877  102    7   16  123   11   19   53   65   11    9  424    2
     3   15   11  190    8   27   91  105   11   33   12   85    8    7
   296  168 1323    7  223  182   16   30   12   20  910    9 3825    2
     5 1343 3904 1159  120    2   61  148]]
[[1877  102    7   16  123   11   19   53   65   11    9  424    2    3
    15   11  190    8   27   91  105   11   33   12   85    8    7  296
   168 1323    7  223  182   16   30   12   20  910    9 3825    2    5
  13

[[3825    2    5 1343 3904 1159  120    2   61  148   16   30   24  365
     8  676    3    7   81  526  105   22   26  245  170   16   56 1081
     2   13   12    1  308  649   18   62 3894    4   15 2090    7  390
     3  294   30   12   13 4898   40   43]]
[[3825    2    5 1343 3904 1159  120    2   61  148   16   30   24  365
     8  676    3    7   81  526  105   22   26  245  170   16   56 1081
     2   13   12    1  308  649   18   62 3894    4   15 2090    7  390
     3  294   30   12   13 4898   40   43]]
[[   2    5 1343 3904 1159  120    2   61  148   16   30   24  365    8
   676    3    7   81  526  105   22   26  245  170   16   56 1081    2
    13   12    1  308  649   18   62 3894    4   15 2090    7  390    3
   294   30   12   13 4898   40   43   10]]
[[   5 1343 3904 1159  120    2   61  148   16   30   24  365    8  676
     3    7   81  526  105   22   26  245  170   16   56 1081    2   13
    12    1  308  649   18   62 3894    4   15 2090    7  390    3  294
    

[[   3  294   30   12   13 4898   40   43   10  935   74  100   98   16
    26   64 1412   15   52  160 5229   53  773   65   81   44   48   60
  1167   23    1 3604  674 2389   18    7   53    3   53    3  226 1089
   771    2 2296   93  166    4   84 1003]]
[[ 294   30   12   13 4898   40   43   10  935   74  100   98   16   26
    64 1412   15   52  160 5229   53  773   65   81   44   48   60 1167
    23    1 3604  674 2389   18    7   53    3   53    3  226 1089  771
     2 2296   93  166    4   84 1003    2]]
[[  30   12   13 4898   40   43   10  935   74  100   98   16   26   64
  1412   15   52  160 5229   53  773   65   81   44   48   60 1167   23
     1 3604  674 2389   18    7   53    3   53    3  226 1089  771    2
  2296   93  166    4   84 1003    2  217]]
[[  12   13 4898   40   43   10  935   74  100   98   16   26   64 1412
    15   52  160 5229   53  773   65   81   44   48   60 1167   23    1
  3604  674 2389   18    7   53    3   53    3  226 1089  771    2 2296
    

...

[[ 352    2   62 3318    4    7 7099    3   93    4   26  494 6574    9
  2252    2   52  235   23    5  113 1029  807   10  218   62  340    1
     9  420   23    5  572 1762   44   52  235   20  112  133   33   12
    13  469  109  151   84  582  337    8]]
[[   2   62 3318    4    7 7099    3   93    4   26  494 6574    9 2252
     2   52  235   23    5  113 1029  807   10  218   62  340    1    9
   420   23    5  572 1762   44   52  235   20  112  133   33   12   13
   469  109  151   84  582  337    8    9]]
[[  62 3318    4    7 7099    3   93    4   26  494 6574    9 2252    2
    52  235   23    5  113 1029  807   10  218   62  340    1    9  420
    23    5  572 1762   44   52  235   20  112  133   33   12   13  469
   109  151   84  582  337    8    9    5]]
[[3318    4    7 7099    3   93    4   26  494 6574    9 2252    2   52
   235   23    5  113 1029  807   10  218   62  340    1    9  420   23
     5  572 1762   44   52  235   20  112  133   33   12   13  469  109
   1

[[ 133   33   12   13  469  109  151   84  582  337    8    9    5   75
    11   30   12  217    3 2210   11 2987   34    7  385   15   11    9
    12   25   29   10  255    4   42   75   71   60  492  194   15  133
   160   19   39  276 3740  102    1 3092]]
[[ 133   33   12   13  469  109  151   84  582  337    8    9    5   75
    11   30   12  217    3 2210   11 2987   34    7  385   15   11    9
    12   25   29   10  255    4   42   75   71   60  492  194   15  133
   160   19   39  276 3740  102    1 3092]]
[[  33   12   13  469  109  151   84  582  337    8    9    5   75   11
    30   12  217    3 2210   11 2987   34    7  385   15   11    9   12
    25   29   10  255    4   42   75   71   60  492  194   15  133  160
    19   39  276 3740  102    1 3092   25]]
[[  12   13  469  109  151   84  582  337    8    9    5   75   11   30
    12  217    3 2210   11 2987   34    7  385   15   11    9   12   25
    29   10  255    4   42   75   71   60  492  194   15  133  160   19
    

[[ 133  160   19   39  276 3740  102    1 3092   25   93  815   44  265
    34    1 2661    4    1   26  119   34   41   11   19  321    1 3168
     4   84 3787    2   42 1326   27 1793    2  157  111   23    7   10
     1  318   11   30   24   35  263 1712]]
[[ 160   19   39  276 3740  102    1 3092   25   93  815   44  265   34
     1 2661    4    1   26  119   34   41   11   19  321    1 3168    4
    84 3787    2   42 1326   27 1793    2  157  111   23    7   10    1
   318   11   30   24   35  263 1712   63]]
[[  19   39  276 3740  102    1 3092   25   93  815   44  265   34    1
  2661    4    1   26  119   34   41   11   19  321    1 3168    4   84
  3787    2   42 1326   27 1793    2  157  111   23    7   10    1  318
    11   30   24   35  263 1712   63   75]]
[[  39  276 3740  102    1 3092   25   93  815   44  265   34    1 2661
     4    1   26  119   34   41   11   19  321    1 3168    4   84 3787
     2   42 1326   27 1793    2  157  111   23    7   10    1  318   11
    

[[  10    1  318   11   30   24   35  263 1712   63   75  239   15  133
     9  601    2  956   35  427   44    7   11    9  325   26  325    7
   250    9 2389   18    5  470   41  168   20    1   53 2877   44   47
   267    4 3912    4  296 4170    4 2336]]
[[  10    1  318   11   30   24   35  263 1712   63   75  239   15  133
     9  601    2  956   35  427   44    7   11    9  325   26  325    7
   250    9 2389   18    5  470   41  168   20    1   53 2877   44   47
   267    4 3912    4  296 4170    4 2336]]
[[   1  318   11   30   24   35  263 1712   63   75  239   15  133    9
   601    2  956   35  427   44    7   11    9  325   26  325    7  250
     9 2389   18    5  470   41  168   20    1   53 2877   44   47  267
     4 3912    4  296 4170    4 2336    3]]
[[ 318   11   30   24   35  263 1712   63   75  239   15  133    9  601
     2  956   35  427   44    7   11    9  325   26  325    7  250    9
  2389   18    5  470   41  168   20    1   53 2877   44   47  267    4
  39

[[ 267    4 3912    4  296 4170    4 2336    3 4763    4 1387    3    8
  1911    7   15   11    9  275   27   86 1068   18  513  489   55 3840
    27   86 1293   17    5  170   20   11   19    1 1062    4  627   15
     7 1360   43   81    3    4   84  300]]
[[   4 3912    4  296 4170    4 2336    3 4763    4 1387    3    8 1911
     7   15   11    9  275   27   86 1068   18  513  489   55 3840   27
    86 1293   17    5  170   20   11   19    1 1062    4  627   15    7
  1360   43   81    3    4   84  300    2]]
[[3912    4  296 4170    4 2336    3 4763    4 1387    3    8 1911    7
    15   11    9  275   27   86 1068   18  513  489   55 3840   27   86
  1293   17    5  170   20   11   19    1 1062    4  627   15    7 1360
    43   81    3    4   84  300    2   92]]
[[   4  296 4170    4 2336    3 4763    4 1387    3    8 1911    7   15
    11    9  275   27   86 1068   18  513  489   55 3840   27   86 1293
    17    5  170   20   11   19    1 1062    4  627   15    7 1360   43
    

In [19]:
print c
pplxty

982


39283.200178849955